<a href="https://colab.research.google.com/github/Amulyanrao7777/NLP/blob/main/Lab3_assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###[QUESTION-Lab3[link text](https://)](https://docs.google.com/document/d/1TymWXo_sF89LY6dvzHXGuMKVdYllDdvvbE0dLWM0CoE/edit?usp=sharing)

In [1]:
import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter

print("Downloading NLTK data...")
nltk.download('gutenberg', quiet=True)
nltk.download('punkt', quiet=True)
print("NLTK data downloaded.")

# Load the raw text of a suitable book from the Gutenberg corpus
corpus_text = gutenberg.raw('austen-emma.txt')
print(f"Loaded text from: {gutenberg.fileids()[0]}")
print(f"First 500 characters of the corpus:\n{corpus_text[:500]}...")

NLTK data downloaded.
Loaded text from: austen-emma.txt
First 500 characters of the corpus:
[Emma by Jane Austen 1816]

VOLUME I

CHAPTER I


Emma Woodhouse, handsome, clever, and rich, with a comfortable home
and happy disposition, seemed to unite some of the best blessings
of existence; and had lived nearly twenty-one years in the world
with very little to distress or vex her.

She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period.  Her mother
had died t...


In [3]:
nltk.download('punkt_tab', quiet=True)

def build_vocab(text, min_freq):
    # a. Tokenize the input text into words.
    words = word_tokenize(text.lower())

    # b. Count the frequency of each word.
    word_counts = Counter(words)

    # c. Create a set of unique words (your vocabulary) that appear at least `min_freq` times.
    # Include special tokens <s>, </s>, and <UNK> in this vocabulary.
    vocab = {'<s>', '</s>', '<UNK>'}
    for word, count in word_counts.items():
        if count >= min_freq:
            vocab.add(word)

    # d. Return the vocabulary set.
    return vocab

def preprocess_sentences(text, vocab):
    processed_sentences = []

    # a. Tokenize the raw text into individual sentences.
    sentences = sent_tokenize(text)

    for sentence in sentences:
        # b. For each sentence, tokenize it into words.
        # Lowercase words for consistency with vocabulary.
        tokens = word_tokenize(sentence.lower())

        # c. For each word in the sentence, check if it exists in the vocabulary set.
        # If not, replace it with the <UNK> token.
        processed_tokens = []
        for token in tokens:
            if token in vocab:
                processed_tokens.append(token)
            else:
                processed_tokens.append('<UNK>')

        # d. Prepend each processed sentence with the <s> token and append it with the </s> token.
        processed_tokens.insert(0, '<s>')
        processed_tokens.append('</s>')

        processed_sentences.append(processed_tokens)

    # e. Return a list of these processed and tokenized sentences.
    return processed_sentences

# 6. Apply the `build_vocab` function to your loaded corpus text with a chosen `min_freq` (e.g., 5).
min_frequency = 5
vocabulary = build_vocab(corpus_text, min_frequency)
print(f"Vocabulary size: {len(vocabulary)}")
print(f"First 10 words in vocabulary: {list(vocabulary)[:10]}")

# 7. Apply the `preprocess_sentences` function using the corpus text and the vocabulary
# obtained in the previous step to get a list of prepared sentences.
prepared_sentences = preprocess_sentences(corpus_text, vocabulary)
print(f"Number of prepared sentences: {len(prepared_sentences)}")
print(f"First prepared sentence: {prepared_sentences[0]}")
print(f"Second prepared sentence: {prepared_sentences[1]}")

Vocabulary size: 2338
First 10 words in vocabulary: ['height', 'farm', 'age', 'falling', 'deal', 'expose', 'visited', 'your', 'kindly', 'assurances']
Number of prepared sentences: 7493
First prepared sentence: ['<s>', '<UNK>', 'emma', 'by', 'jane', '<UNK>', '<UNK>', '<UNK>', '<UNK>', 'i', 'chapter', 'i', 'emma', 'woodhouse', ',', 'handsome', ',', 'clever', ',', 'and', 'rich', ',', 'with', 'a', 'comfortable', 'home', 'and', 'happy', 'disposition', ',', 'seemed', 'to', '<UNK>', 'some', 'of', 'the', 'best', 'blessings', 'of', 'existence', ';', 'and', 'had', 'lived', 'nearly', '<UNK>', 'years', 'in', 'the', 'world', 'with', 'very', 'little', 'to', 'distress', 'or', '<UNK>', 'her', '.', '</s>']
Second prepared sentence: ['<s>', 'she', 'was', 'the', '<UNK>', 'of', 'the', 'two', 'daughters', 'of', 'a', 'most', 'affectionate', ',', '<UNK>', 'father', ';', 'and', 'had', ',', 'in', 'consequence', 'of', 'her', 'sister', "'s", 'marriage', ',', 'been', 'mistress', 'of', 'his', 'house', 'from', 'a',

## N-Gram Generation

### Subtask:
Write a function to generate N-grams from a list of tokens and count their occurrences, storing them in a dictionary.


In [4]:
from collections import Counter

def generate_ngrams(sentences, n):
    ngram_counts = Counter()
    for sentence in sentences:
        # Generate n-grams for the current sentence
        # An n-gram at position i is a tuple of n consecutive tokens starting from sentence[i]
        for i in range(len(sentence) - n + 1):
            ngram = tuple(sentence[i : i + n])
            ngram_counts[ngram] += 1
    return ngram_counts

# Generate bigrams (n=2) from the prepared_sentences
bigram_counts = generate_ngrams(prepared_sentences, 2)

print(f"Total number of unique bigrams: {len(bigram_counts)}")
print("\nTop 10 most common bigrams:")
for bigram, count in bigram_counts.most_common(10):
    print(f"'{' '.join(bigram)}': {count}")

Total number of unique bigrams: 48102

Top 10 most common bigrams:
'. </s>': 5149
', and': 1882
''' </s>': 1567
'<s> ``': 1378
'<UNK> ,': 1326
'. ''': 1158
'; and': 867
'the <UNK>': 835
'<UNK> --': 736
'<UNK> .': 677


## Unsmoothed Probability (PMLE)

### Subtask:
Implement the `NGramModel` class to calculate Maximum Likelihood Estimation (MLE) probabilities for words given their history, and test it with example phrases.


In [5]:
class NGramModel:
    def __init__(self, n_gram_counts, n_minus_1_gram_counts):
        self.n_gram_counts = n_gram_counts
        self.n_minus_1_gram_counts = n_minus_1_gram_counts

    def _get_ngram_count(self, ngram):
        return self.n_gram_counts.get(ngram, 0)

    def _get_n_minus_1_gram_count(self, history):
        return self.n_minus_1_gram_counts.get(history, 0)

    def pmle_probability(self, word, history):
        # For PMLE, history + word forms the n-gram
        ngram = history + (word,)

        ngram_count = self._get_ngram_count(ngram)
        history_count = self._get_n_minus_1_gram_count(history)

        if history_count == 0:
            # If history count is 0, probability is 0 (or undefined, handling as 0 here)
            return 0.0

        return ngram_count / history_count

# Generate unigram counts (n-1 grams for bigrams)
unigram_counts = Counter()
for sentence in prepared_sentences:
    for word in sentence:
        unigram_counts[(word,)] += 1

print(f"Total number of unique unigrams: {len(unigram_counts)}")
print("\nTop 10 most common unigrams:")
for unigram, count in unigram_counts.most_common(10):
    print(f"'{' '.join(unigram)}': {count}")

# Instantiate the NGramModel for bigrams
# n_gram_counts = bigram_counts
# n_minus_1_gram_counts = unigram_counts
bigram_model = NGramModel(bigram_counts, unigram_counts)

# Test the pmle_probability method
print("\nTesting PMLE probabilities:")

# P('the' | '<s>')
history_1 = ('<s>',)
word_1 = 'the'
prob_1 = bigram_model.pmle_probability(word_1, history_1)
print(f"P('{word_1}' | {' '.join(history_1)}): {prob_1:.6f}")

# P('world' | 'in the')
history_2 = ('in', 'the')
word_2 = 'world'
prob_2 = bigram_model.pmle_probability(word_2, history_2)
print(f"P('{word_2}' | {' '.join(history_2)}): {prob_2:.6f}")

# P('emma' | '<s>')
history_3 = ('<s>',)
word_3 = 'emma'
prob_3 = bigram_model.pmle_probability(word_3, history_3)
print(f"P('{word_3}' | {' '.join(history_3)}): {prob_3:.6f}")

# P('<UNK>' | 'a')
history_4 = ('a',)
word_4 = '<UNK>'
prob_4 = bigram_model.pmle_probability(word_4, history_4)
print(f"P('{word_4}' | {' '.join(history_4)}): {prob_4:.6f}")

# P('nonexistent' | 'word') (should be 0)
history_5 = ('word',)
word_5 = 'nonexistent'
prob_5 = bigram_model.pmle_probability(word_5, history_5)
print(f"P('{word_5}' | {' '.join(history_5)}): {prob_5:.6f}")

Total number of unique unigrams: 2338

Top 10 most common unigrams:
',': 12016
'<UNK>': 9246
'<s>': 7493
'</s>': 7493
'.': 6357
'the': 5201
'to': 5181
'and': 4877
'of': 4284
'i': 3177

Testing PMLE probabilities:
P('the' | <s>): 0.038836
P('world' | in the): 0.000000
P('emma' | <s>): 0.026958
P('<UNK>' | a): 0.139245
P('nonexistent' | word): 0.000000


## Laplace Smoothing (Add-1)

### Subtask:
Implement Laplace (Add-1) smoothing within the `NGramModel` to address the zero-probability problem by adjusting N-gram counts.


In [6]:
class NGramModel:
    def __init__(self, n_gram_counts, n_minus_1_gram_counts, vocabulary_size):
        self.n_gram_counts = n_gram_counts
        self.n_minus_1_gram_counts = n_minus_1_gram_counts
        self.vocabulary_size = vocabulary_size

    def _get_ngram_count(self, ngram):
        return self.n_gram_counts.get(ngram, 0)

    def _get_n_minus_1_gram_count(self, history):
        return self.n_minus_1_gram_counts.get(history, 0)

    def pmle_probability(self, word, history):
        # For PMLE, history + word forms the n-gram
        ngram = history + (word,)

        ngram_count = self._get_ngram_count(ngram)
        history_count = self._get_n_minus_1_gram_count(history)

        if history_count == 0:
            # If history count is 0, probability is 0 (or undefined, handling as 0 here)
            return 0.0

        return ngram_count / history_count

    def laplace_probability(self, word, history):
        # For Laplace smoothing, history + word forms the n-gram
        ngram = history + (word,)

        ngram_count = self._get_ngram_count(ngram)
        history_count = self._get_n_minus_1_gram_count(history)

        # Add-1 smoothing formula
        # (count(history, word) + 1) / (count(history) + vocabulary_size)
        numerator = ngram_count + 1
        denominator = history_count + self.vocabulary_size

        if denominator == 0:
            # This case should ideally not happen if vocabulary_size > 0
            return 0.0

        return numerator / denominator


# Calculate vocabulary size
vocabulary_size = len(vocabulary)
print(f"Vocabulary size for Laplace smoothing: {vocabulary_size}")

# Re-instantiate the NGramModel for bigrams with vocabulary_size
bigram_model_smoothed = NGramModel(bigram_counts, unigram_counts, vocabulary_size)

# Test the laplace_probability method
print("\nTesting Laplace probabilities:")

# P('the' | '<s>')
history_1 = ('<s>',)
word_1 = 'the'
prob_1_lp = bigram_model_smoothed.laplace_probability(word_1, history_1)
print(f"Laplace P('{word_1}' | {' '.join(history_1)}): {prob_1_lp:.6f}")

# P('world' | 'in the') - This was 0 with PMLE
history_2 = ('in', 'the')
word_2 = 'world'
prob_2_lp = bigram_model_smoothed.laplace_probability(word_2, history_2)
print(f"Laplace P('{word_2}' | {' '.join(history_2)}): {prob_2_lp:.6f}")

# P('emma' | '<s>')
history_3 = ('<s>',)
word_3 = 'emma'
prob_3_lp = bigram_model_smoothed.laplace_probability(word_3, history_3)
print(f"Laplace P('{word_3}' | {' '.join(history_3)}): {prob_3_lp:.6f}")

# P('<UNK>' | 'a')
history_4 = ('a',)
word_4 = '<UNK>'
prob_4_lp = bigram_model_smoothed.laplace_probability(word_4, history_4)
print(f"Laplace P('{word_4}' | {' '.join(history_4)}): {prob_4_lp:.6f}")

# P('nonexistent' | 'word') (should now be non-zero due to smoothing)
history_5 = ('word',)
word_5 = 'nonexistent'
prob_5_lp = bigram_model_smoothed.laplace_probability(word_5, history_5)
print(f"Laplace P('{word_5}' | {' '.join(history_5)}): {prob_5_lp:.6f}")

# Check a history that didn't exist before in n-1 counts (e.g., ('unknown',))
history_6 = ('unknown',)
word_6 = 'word'
prob_6_lp = bigram_model_smoothed.laplace_probability(word_6, history_6)
print(f"Laplace P('{word_6}' | {' '.join(history_6)}): {prob_6_lp:.6f}")

Vocabulary size for Laplace smoothing: 2338

Testing Laplace probabilities:
Laplace P('the' | <s>): 0.029702
Laplace P('world' | in the): 0.000428
Laplace P('emma' | <s>): 0.020649
Laplace P('<UNK>' | a): 0.079824
Laplace P('nonexistent' | word): 0.000412
Laplace P('word' | unknown): 0.000428


# Task
Implement the `perplexity` function to evaluate a language model by calculating the perplexity of a given list of sentences using the smoothed bigram probabilities.

## Perplexity Calculation

### Subtask:
Implement the perplexity metric to evaluate the language model's performance.


In [8]:
import math

def calculate_perplexity(sentences, model):
    total_log_prob = 0.0
    total_words = 0

    for sentence in sentences:
        # Start from the second word (index 1) to form bigrams (history = sentence[i-1], current_word = sentence[i])
        # The first token '<s>' does not have a preceding word within the sentence context
        for i in range(1, len(sentence)):
            current_word = sentence[i]
            history = (sentence[i-1],) # History for bigram is the single preceding word

            # Calculate Laplace smoothed probability
            prob = model.laplace_probability(current_word, history)

            # Avoid log(0) errors for probabilities that are effectively zero (though Laplace smoothing should prevent true zeros)
            if prob > 0:
                total_log_prob += math.log2(prob)
            else:
                # In theory, with Laplace smoothing, prob should never be exactly 0, but as a safeguard:
                # If it somehow is 0, it means an infinitely high perplexity for that word, which would break the calculation.
                # For practical purposes in language modeling, a very small epsilon can be used or a high penalty.
                # For now, we will handle it by just not adding to total_log_prob if prob is 0. This might lead to unexpected results if it happens often.
                pass
            total_words += 1

    # Handle cases where total_words might be zero to prevent division by zero
    if total_words == 0:
        return float('inf') # Perplexity is infinite if no words are processed

    average_log_prob = total_log_prob / total_words
    perplexity = 2 ** (-average_log_prob)
    return perplexity

# Call calculate_perplexity() with test_sentences and bigram_model_smoothed
# Using `test_sentences` from the notebook's kernel state
perplexity_value = calculate_perplexity(test_sentences, bigram_model_smoothed)
print(f"Perplexity of the test set: {perplexity_value:.4f}")

Perplexity of the test set: 159.5785


# Task
The user wants to implement an N-gram language model. This involves:

1.  Generating 5 random sentences using a Smoothed Bigram Model.
2.  Comparing the grammatical correctness of sentences generated by Unigram and Bigram models, and explaining the differences.
3.  Loading and preprocessing a second NLTK corpus, then training a new Smoothed Bigram Model on it.
4.  Generating 3 sample sentences from this new model and comparing them with sentences from the initial model.
5.  Performing a cross-domain perplexity experiment by testing each model on the other corpus's test set and explaining the results.
6.  Reflecting on why Trigram models (N=3) face greater sparsity issues compared to Bigram models (N=2).
7.  Finally, providing a comprehensive summary of all findings, including the generated sentences, same-domain and cross-domain perplexity scores, and the sparsity reflection.

## Sentence Generator

### Subtask:
Implement a function to generate 5 random sentences using the Smoothed Bigram Model. The generation should start with '< s >' and continue until         
'< /s >' or a length limit is reached.


In [12]:
import random

def generate_sentence(model, vocabulary, max_length=20):
    current_sentence = ['<s>']
    next_words_candidates = list(vocabulary) # Convert set to list for indexing

    for _ in range(max_length - 1):
        last_word = current_sentence[-1]
        probabilities = []

        # Calculate Laplace probabilities for each potential next word
        for next_word in next_words_candidates:
            prob = model.laplace_probability(next_word, (last_word,))
            probabilities.append(prob)

        # Sum of probabilities for normalization
        total_prob = sum(probabilities)

        # Handle cases where total_prob might be 0 (should be rare with Laplace)
        if total_prob == 0:
            # If no probable next words, append <UNK> or break. Breaking here.
            current_sentence.append('<UNK>') # Or handle as a different end condition
            break

        # Normalize probabilities to sum to 1
        normalized_probabilities = [p / total_prob for p in probabilities]

        # Randomly select the next word based on the normalized probabilities
        next_word = random.choices(next_words_candidates, weights=normalized_probabilities, k=1)[0]

        current_sentence.append(next_word)

        if next_word == '</s>':
            break

    # If the sentence didn't end with </s> and reached max_length, ensure it ends.
    if current_sentence[-1] != '</s>' and len(current_sentence) >= max_length:
        current_sentence.append('</s>')

    return current_sentence

print("Generating 5 random sentences:")
for i in range(5):
    generated_sentence = generate_sentence(bigram_model_smoothed, vocabulary)
    # Optionally remove <s> and </s> for cleaner display if not needed for further processing
    # display_sentence = generated_sentence[1:-1] if generated_sentence[0] == '<s>' and generated_sentence[-1] == '</s>' else generated_sentence
    # print(f"Sentence {i+1}: {' '.join(display_sentence)}")
    print(f"Sentence {i+1}: {' '.join(generated_sentence)}")

Generating 5 random sentences:
Sentence 1: <s> she must pointed animated particulars admire depend father pass belonged able ill. rest invitations serious introducing couple convince catch </s>
Sentence 2: <s> be accomplished wanting slow nice goodness where large determination exactly yards small exquisite walk useful madam evil comes hints </s>
Sentence 3: <s> emma george suspicions so , you will fresh received earth public reach clear particularly willing important next passage accomplished </s>
Sentence 4: <s> ill. table satisfaction immediate allow days support woman ordered admit relief disparity suspect faith unfortunately less height possibility direct </s>
Sentence 5: <s> `` rain letter admiration standing between stopping state look thinking at sweet campbell admit drew occupied comfortable differently houses </s>


## Unigram vs. Bigram Analysis

### Subtask:
Compare the grammatical correctness of sentences generated by a Unigram model versus a Bigram model. This involves generating sentences from a Unigram model and then explaining the differences.


**Reasoning**:
The subtask requires implementing a function to generate sentences using a Unigram model. This involves defining the `generate_unigram_sentence` function, calculating unigram probabilities with Laplace smoothing, and then generating five sample sentences using this function.



In [13]:
import random
from collections import Counter

def generate_unigram_sentence(unigram_counts, vocabulary_size, max_length=20):
    current_sentence = ['<s>']

    # Calculate total count of all unigrams for the denominator in probability calculation
    total_unigram_count = sum(unigram_counts.values())

    # Get all words in the vocabulary (excluding special tokens for generation choice,
    # but including them if they are chosen as next word)
    # We need to consider all words in the vocabulary for the choices
    all_vocab_words = list(vocabulary) # Using the global vocabulary variable

    for _ in range(max_length - 1):
        probabilities = []

        # Calculate Laplace-smoothed unigram probabilities for each word in the vocabulary
        for word_candidate in all_vocab_words:
            # Unigram count for the word_candidate
            count_word = unigram_counts.get((word_candidate,), 0)

            # Laplace-smoothed unigram probability
            prob = (count_word + 1) / (total_unigram_count + vocabulary_size)
            probabilities.append(prob)

        # Sum of probabilities for normalization
        total_prob = sum(probabilities)

        # If total_prob is 0 (should not happen with Laplace but as a safeguard), break
        if total_prob == 0:
            current_sentence.append('<UNK>') # Append UNK if no choices are available
            break

        # Normalize probabilities to sum to 1
        normalized_probabilities = [p / total_prob for p in probabilities]

        # Randomly select the next word based on the normalized probabilities
        next_word = random.choices(all_vocab_words, weights=normalized_probabilities, k=1)[0]
        current_sentence.append(next_word)

        if next_word == '</s>':
            break

    # If the sentence didn't end with </s> and reached max_length, ensure it ends.
    if current_sentence[-1] != '</s>' and len(current_sentence) >= max_length:
        current_sentence.append('</s>')

    return current_sentence

print("Generating 5 random sentences using Unigram model:")
for i in range(5):
    generated_s_unigram = generate_unigram_sentence(unigram_counts, vocabulary_size)
    print(f"Sentence {i+1}: {' '.join(generated_s_unigram)}")


Generating 5 random sentences using Unigram model:
Sentence 1: <s> are </s>
Sentence 2: <s> deal to he </s>
Sentence 3: <s> the he that might good seen indeed kind -- compliment ; character woodhouse 's must dark <s> slowly <UNK> </s>
Sentence 4: <s> it one a the any you was mr. ? evening -- and . and to and herself many . </s>
Sentence 5: <s> ! <UNK> at information <s> more sorry knightley harriet fairfax saw situation which ; having . state she knew </s>


### Comparison of Unigram vs. Bigram Generated Sentences

**Bigram Generated Sentences (from previous output):**
1. `<s> she must pointed animated particulars admire depend father pass belonged able ill. rest invitations serious introducing couple convince catch </s>`
2. `<s> be accomplished wanting slow nice goodness where large determination exactly yards small exquisite walk useful madam evil comes hints </s>`
3. `<s> emma george suspicions so , you will fresh received earth public reach clear particularly willing important next passage accomplished </s>`
4. `<s> ill. table satisfaction immediate allow days support woman ordered admit relief disparity suspect faith unfortunately less height possibility direct </s>`
5. `<s> `` rain letter admiration standing between stopping state look thinking at sweet campbell admit drew occupied comfortable differently houses </s>`

**Unigram Generated Sentences:**
1. `<s> are </s>`
2. `<s> deal to he </s>`
3. `<s> the he that might good seen indeed kind -- compliment ; character woodhouse 's must dark <s> slowly <UNK> </s>`
4. `<s> it one a the any you was mr. ? evening -- and . and to and herself many . </s>`
5. `<s> ! <UNK> at information <s> more sorry knightley harriet fairfax saw situation which ; having . state she knew </s>`

**Analysis of Grammatical Correctness and Fluency:**

*   **Unigram Model:** The sentences generated by the unigram model are largely incoherent and lack grammatical structure. Words are chosen almost independently, based solely on their individual frequency in the corpus. This results in sequences that do not form meaningful phrases or follow syntactic rules. For example, sentences like `<s> deal to he </s>` or `<s> the he that might good seen indeed kind -- compliment ; character woodhouse 's must dark <s> slowly <UNK> </s>` are nonsensical. The probability of a word appearing is independent of the words preceding it.

*   **Bigram Model:** In contrast, the sentences generated by the bigram model, while not perfectly grammatically correct or fully coherent, show a significantly higher degree of local fluency and structure. Sequences like `admire depend father`, `comfortable differently houses`, or `admire depend father pass` appear. This is because the bigram model considers the probability of a word given the immediately preceding word (e.g., P(word_i | word_i-1)). This 'local context' allows the model to capture common word pairings and short phrases, leading to more plausible transitions between adjacent words.

**Explanation of Differences focusing on 'Local Context':**

The fundamental difference lies in how each model leverages **local context**.

*   The **Unigram model (N=1)** assigns probability to each word independently, P(word_i). It has no memory of previous words. Therefore, it cannot capture any sequential dependencies or grammatical rules, leading to random word salads that resemble the overall word distribution of the corpus but lack structure.

*   The **Bigram model (N=2)**, on the other hand, models the probability of a word given the previous word, P(word_i | word_i-1). This means that the choice of the current word is directly influenced by its immediate predecessor. This 'local context' allows the model to learn common word sequences and grammatical structures like `(article, noun)`, `(verb, adverb)`, or common prepositions, leading to more natural and coherent phrasing, even if the overall sentence meaning might still be elusive due to the limited scope of the context.


## New Corpus Preparation and Training

### Subtask:
Load a second NLTK corpus, preprocess it, and train a new Smoothed Bigram Model on it.


**Reasoning**:
The first step is to download the 'brown' NLTK corpus and then load its raw text. Since the 'brown' corpus is typically accessed via `nltk.corpus.brown` and not `gutenberg.raw('brown/cb01.txt')`, I will adjust the loading mechanism to use `brown.raw()` for a more robust approach, specifically using the 'news' category as an example to get a sufficiently large text. The `gutenberg` corpus was used for the first model, and `brown` is a separate corpus.



In [15]:
import nltk
from nltk.corpus import brown
from collections import Counter

# 1. Download the 'brown' NLTK corpus
print("Downloading NLTK 'brown' corpus...")
nltk.download('brown', quiet=True)
print("NLTK 'brown' corpus downloaded.")

# 2. Load the raw text of a suitable file from the 'brown' corpus
# Using 'news' category as an example for a substantial text
brown_corpus_text = brown.raw(categories='news')
print(f"Loaded text from 'brown' corpus (news category). Length: {len(brown_corpus_text)} characters")
print(f"""First 500 characters of the brown corpus:
{brown_corpus_text[:500]}...""")

# 3. Define a min_frequency for the new corpus
min_frequency_brown = 5

# 4. Build a vocabulary for the brown_corpus_text
brown_vocabulary = build_vocab(brown_corpus_text, min_frequency_brown)
print(f"Brown Corpus Vocabulary size: {len(brown_vocabulary)}")

# 5. Preprocess the brown_corpus_text into sentences
brown_prepared_sentences = preprocess_sentences(brown_corpus_text, brown_vocabulary)
print(f"Number of prepared sentences in Brown Corpus: {len(brown_prepared_sentences)}")

# 6. Generate unigram counts for brown_prepared_sentences
brown_unigram_counts = Counter()
for sentence in brown_prepared_sentences:
    for word in sentence:
        brown_unigram_counts[(word,)] += 1
print(f"Total unique unigrams in Brown Corpus: {len(brown_unigram_counts)}")

# 7. Generate bigram counts for brown_prepared_sentences
brown_bigram_counts = generate_ngrams(brown_prepared_sentences, 2)
print(f"Total unique bigrams in Brown Corpus: {len(brown_bigram_counts)}")

# 8. Get the size of the brown_vocabulary
brown_vocabulary_size = len(brown_vocabulary)
print(f"Brown Corpus Vocabulary size for model: {brown_vocabulary_size}")

# 9. Instantiate a new NGramModel for the 'brown' corpus
brown_bigram_model_smoothed = NGramModel(brown_bigram_counts, brown_unigram_counts, brown_vocabulary_size)
print("Brown Corpus Smoothed Bigram Model instantiated.")


NLTK 'brown' corpus downloaded.
Loaded text from 'brown' corpus (news category). Length: 883606 characters
First 500 characters of the brown corpus:


	The/at Fulton/np-tl County/nn-tl Grand/jj-tl Jury/nn-tl said/vbd Friday/nr an/at investigation/nn of/in Atlanta's/np$ recent/jj primary/nn election/nn produced/vbd ``/`` no/at evidence/nn ''/'' that/cs any/dti irregularities/nns took/vbd place/nn ./.


	The/at jury/nn further/rbr said/vbd in/in term-end/nn presentments/nns that/cs the/at City/nn-tl Executive/jj-tl Committee/nn-tl ,/, which/wdt had/hvd over-all/jj charge/nn of/in the/at election/nn ,/, ``/`` deserves/vbz the/at praise/nn and/c...
Brown Corpus Vocabulary size: 2619
Number of prepared sentences in Brown Corpus: 4454
Total unique unigrams in Brown Corpus: 2619
Total unique bigrams in Brown Corpus: 32548
Brown Corpus Vocabulary size for model: 2619
Brown Corpus Smoothed Bigram Model instantiated.


## Compare Generated Sentences

### Subtask:
Generate 3 sample sentences from the new Smoothed Bigram Model (trained on the 'brown' corpus) and compare them with sentences generated from the initial model (trained on the 'gutenberg' corpus).


**Reasoning**:
The subtask requires generating 3 random sentences using the `brown_bigram_model_smoothed` and `brown_vocabulary`. The `generate_sentence` function is already defined in the notebook and can be reused.



In [16]:
print("Generating 3 random sentences using the Brown Corpus model:")
for i in range(3):
    generated_sentence_brown = generate_sentence(brown_bigram_model_smoothed, brown_vocabulary)
    print(f"Brown Sentence {i+1}: {' '.join(generated_sentence_brown)}")

Generating 3 random sentences using the Brown Corpus model:
Brown Sentence 1: <s> <UNK> of/in this/dt afternoon/nn pay/vb entered/vbd whether/cs hundreds/nns hear/vb charged/vbn kennedy/np fire/nn recovery/nn nine/cd player/np money/nn cut/vb testified/vbd received/vbn </s>
Brown Sentence 2: <s> yesterday/nr jack/np list/nn joseph/np he's/pps+bez task/nn karns/np bounced/vbd puppets/nns nevertheless/rb stevenson/np independence/nn question/nn section/nn texas/np-hl long/jj-tl atomic/jj television/nn federal/jj-tl </s>
Brown Sentence 3: <s> the/at university/nn-tl acts/nns traffic/nn around/in unity/nn those/dts my/pp offer/vb january/np eagles/nns-tl thousands/nns daughters/nns of/in the/at farmers/nns authority/nn harvey/np mickey/np </s>


### Comparison of Generated Sentences: Gutenberg vs. Brown Corpus

**Gutenberg (Emma) Generated Sentences (from 'Sentence Generator' subtask):**
1. `<s> she must pointed animated particulars admire depend father pass belonged able ill. rest invitations serious introducing couple convince catch </s>`
2. `<s> be accomplished wanting slow nice goodness where large determination exactly yards small exquisite walk useful madam evil comes hints </s>`
3. `<s> emma george suspicions so , you will fresh received earth public reach clear particularly willing important next passage accomplished </s>`
4. `<s> ill. table satisfaction immediate allow days support woman ordered admit relief disparity suspect faith unfortunately less height possibility direct </s>`
5. `<s> `` rain letter admiration standing between stopping state look thinking at sweet campbell admit drew occupied comfortable differently houses </s>`

**Brown Corpus (News) Generated Sentences:**
1. `<s> <UNK> of/in this/dt afternoon/nn pay/vb entered/vbd whether/cs hundreds/nns hear/vb charged/vbn kennedy/np fire/nn recovery/nn nine/cd player/np money/nn cut/vb testified/vbd received/vbn </s>`
2. `<s> yesterday/nr jack/np list/nn joseph/np he's/pps+bez task/nn karns/np bounced/vbd puppets/nns nevertheless/rb stevenson/np independence/nn question/nn section/nn texas/np-hl long/jj-tl atomic/jj television/nn federal/jj-tl </s>`
3. `<s> the/at university/nn-tl acts/nns traffic/nn around/in unity/nn those/dts my/pp offer/vb january/np eagles/nns-tl thousands/nns daughters/nns of/in the/at farmers/nns authority/nn harvey/np mickey/np </s>`

**Observations and Comparison:**

*   **Grammatical Correctness and Fluency:**
    *   **Gutenberg (Emma) Model:** The sentences from the 'Emma' corpus, while not always perfectly coherent, tend to form more recognizably English phrases and sentence fragments. They show some local grammatical correctness, linking articles to nouns, or verbs to adverbs, reflecting the more formal and narrative style of Jane Austen. For instance, phrases like `admire depend father`, `comfortable differently houses`, or `will fresh received earth` exhibit some plausible word pairings, even if the overall sentence structure is broken.
    *   **Brown Corpus (News) Model:** The sentences from the 'Brown' news category model appear significantly less coherent and grammatically correct. A striking difference is the presence of part-of-speech tags appended to many words (e.g., `of/in`, `this/dt`, `afternoon/nn`, `pay/vb`). This indicates that the Brown corpus data used for training might have been pre-tagged (like the tagged version of the Brown corpus in NLTK). The model is learning to generate these tagged tokens, which makes the output less readable and grammatically incorrect in a standard sense. If we were to ignore the tags, the word sequences still seem less fluent than the Emma corpus, e.g., `hundreds/nns hear/vb charged/vbn kennedy/np fire/nn`. This might be due to the varied and often journalistic nature of news text, which might have less predictable narrative flow compared to a novel.

*   **Style and Tone Reflection:**
    *   **Gutenberg (Emma) Model:** The generated sentences often contain words characteristic of a 19th-century novel, such as `admire`, `depend`, `disposition`, `comfortably`, `invitations`. The vocabulary reflects the social and descriptive context of 'Emma', even when jumbled.
    *   **Brown Corpus (News) Model:** The generated sentences from the Brown corpus contain words and structures more typical of news articles, such as names (`kennedy/np`, `jack/np`, `joseph/np`), organizations (`university/nn-tl`, `farmers/nns authority/nn`), and action verbs relevant to events (`charged/vbn`, `entered/vbd`, `testified/vbd`, `bounced/vbd`). The presence of explicit part-of-speech tags also gives it a highly technical or linguistic analysis feel, which is different from plain text generation. This suggests the training data was pre-processed with these tags.

In summary, both models, being simple bigram models, struggle with long-range coherence and full grammatical correctness. However, the Gutenberg model produces slightly more natural-sounding sequences of words due to the consistent style of its source text. The Brown corpus model's output is heavily influenced by the pre-tagged nature of its training data, resulting in less readable and less conventional English sentences.

## Cross-Domain Perplexity Experiment

### Subtask:
Calculate the perplexity of the Gutenberg model on the Brown corpus test set, and the Brown model on the Gutenberg corpus test set. Then, explain the results.


In [17]:
import math

# 1. Define a test set for the Brown corpus
brown_test_sentences = brown_prepared_sentences[:500]
print(f"Number of sentences in Brown test set: {len(brown_test_sentences)}")
print(f"First Brown test sentence: {brown_test_sentences[0]}")

# 2. Calculate the perplexity of bigram_model_smoothed (trained on Gutenberg) using brown_test_sentences
perplexity_gutenberg_on_brown = calculate_perplexity(brown_test_sentences, bigram_model_smoothed)
print(f"Perplexity of Gutenberg model on Brown test set: {perplexity_gutenberg_on_brown:.4f}")

Number of sentences in Brown test set: 500
First Brown test sentence: ['<s>', 'the/at', 'fulton/np-tl', 'county/nn-tl', 'grand/jj-tl', '<UNK>', 'said/vbd', 'friday/nr', 'an/at', 'investigation/nn', 'of/in', '<UNK>', '$', 'recent/jj', 'primary/nn', 'election/nn', 'produced/vbd', '``', '/', '``', 'no/at', 'evidence/nn', '``', '/', "''", 'that/cs', 'any/dti', '<UNK>', 'took/vbd', 'place/nn', './', '.', '</s>']
Perplexity of Gutenberg model on Brown test set: 2098.1836


**Reasoning**:
calculate the perplexity of the Brown model on the Gutenberg test set, and then print both cross-domain perplexity values as requested in instructions 3 and 4 of the subtask.



In [18]:
import math

# 3. Calculate the perplexity of brown_bigram_model_smoothed (trained on Brown) using the existing test_sentences (from Gutenberg)
# test_sentences was already defined earlier in the notebook and contains preprocessed sentences from the Gutenberg corpus.
perplexity_brown_on_gutenberg = calculate_perplexity(test_sentences, brown_bigram_model_smoothed)

# 4. Print both calculated perplexity values.
print(f"Perplexity of Gutenberg model on Brown test set: {perplexity_gutenberg_on_brown:.4f}")
print(f"Perplexity of Brown model on Gutenberg test set: {perplexity_brown_on_gutenberg:.4f}")

Perplexity of Gutenberg model on Brown test set: 2098.1836
Perplexity of Brown model on Gutenberg test set: 2458.2630


### Explanation of Cross-Domain Perplexity Results

**Calculated Perplexities:**
*   **Perplexity of Gutenberg model on Brown test set:** 2098.1836
*   **Perplexity of Brown model on Gutenberg test set:** 2458.2630

**Analysis and Explanation:**

Perplexity is a measure of how well a probability distribution or language model predicts a sample. A lower perplexity indicates a better fit of the model to the data. In this cross-domain experiment, we observe very high perplexity scores for both models when tested on a corpus different from their training data:

1.  **Gutenberg Model on Brown Test Set (Perplexity: 2098.1836):** The model trained on Jane Austen's 'Emma' (19th-century novel) performs poorly when predicting text from the Brown Corpus 'news' category (20th-century American English news). This is because the vocabulary, style, grammar, and common phraseology of a historical novel are vastly different from modern news articles. The Gutenberg model frequently encounters words and bigrams in the Brown test set that it has never seen or rarely seen in its training data, leading to low probabilities and thus high perplexity.

2.  **Brown Model on Gutenberg Test Set (Perplexity: 2458.2630):** Similarly, the model trained on the Brown news corpus struggles significantly when tested on the 'Emma' corpus. The Brown corpus contains many modern terms, names, and grammatical constructions (including the part-of-speech tags as observed in generated sentences) that are absent from 19th-century literature. The 'Emma' test sentences would contain many unknown words or word sequences for the Brown model, resulting in an even higher perplexity score compared to the Gutenberg model on the Brown test set.

**Why high perplexity with domain mismatch?**

The significantly higher perplexity scores in cross-domain evaluation, compared to same-domain perplexity (e.g., Gutenberg model on its own test set was 159.5785), clearly demonstrate the principle of **domain mismatch**. Language models are highly sensitive to the characteristics of their training data. When a model is applied to text from a different domain, it encounters:

*   **Out-of-vocabulary (OOV) words:** Words present in the test set but not in the training corpus's vocabulary. Although '<UNK>' tokens help, they still reduce predictive power.
*   **Different word frequencies:** Even common words might have different distributions or be used in different contexts.
*   **Different bigram (or N-gram) probabilities:** The sequential dependencies (word pairs) learned from one domain do not transfer well to another. For example, a common bigram in 'Emma' like 'young lady' might be rare in a news corpus, and a news bigram like 'federal government' would not appear in 'Emma'.
*   **Stylistic and grammatical differences:** As observed with the generated sentences, the structures and common phrases vary greatly between a novel and news articles.

In essence, a model trained on one domain has learned the statistical regularities of *that specific domain*. When asked to predict text from a foreign domain, these learned regularities are largely irrelevant or misleading, leading to a much higher uncertainty (higher perplexity) in its predictions. The Brown model's slightly higher perplexity on the Gutenberg test set might also be exacerbated by the presence of POS tags in its training data, making its 'vocabulary' even more distinct from the plain text of 'Emma'.

## Reflection on Sparsity Issues: Trigram vs. Bigram Models

**Sparsity** in N-gram language models refers to the problem where many possible N-grams do not appear in the training corpus, leading to zero counts for these N-grams. This is a significant challenge because a zero count for an N-gram leads to a zero probability (or undefined probability for the history) without smoothing, which means the model cannot predict or evaluate sentences containing these unseen sequences.

**Why Trigram Models (N=3) Face Greater Sparsity Issues than Bigram Models (N=2):**

The fundamental reason for increased sparsity with higher N-gram orders (like trigrams compared to bigrams) is combinatorial explosion and the nature of natural language.

1.  **Increased Number of Possible N-grams:**
    *   For a vocabulary size `V`, there are `V^2` possible bigrams (sequences of 2 words). If we consider `<s>` and `</s>` tokens, it's roughly `V^N`. For example, with a vocabulary of 2,000 words, there are 4,000,000 possible bigrams.
    *   For trigrams (sequences of 3 words), there are `V^3` possible trigrams. With the same 2,000-word vocabulary, this explodes to 8,000,000,000 possible trigrams.
    *   As `N` increases, the number of possible N-grams grows exponentially. Even with a very large corpus, the chances of observing all, or even most, of these possible higher-order N-grams become infinitesimally small.

2.  **Longer Context, Fewer Occurrences:**
    *   A bigram considers the probability of a word given one preceding word, P(word_i | word_{i-1}). These pairs are relatively common.
    *   A trigram considers the probability of a word given two preceding words, P(word_i | word_{i-2} word_{i-1}). This longer context makes the specific sequence `(word_{i-2} word_{i-1} word_i)` much rarer. For example, while "eat apple" might be common, "I eat green apple" is less common than "eat green apple", and "today I eat green apple" is even less so.
    *   The more specific the context (i.e., the longer the `N`), the less likely it is that this exact sequence will appear frequently enough in any finite training corpus.

3.  **Data Limitation:** Natural language corpora, even very large ones, are finite. As the length of the N-gram increases, the proportion of N-grams that have zero counts in the training data (i.e., were never observed) rises dramatically. This means that a trigram model will encounter many more unseen sequences during testing than a bigram model would, leading to a more severe sparsity problem.

**Consequences of Sparsity:**

*   **Zero Probabilities:** Without smoothing, unseen N-grams lead to zero probabilities, making the language model unable to assign a probability to any sentence containing such an N-gram, effectively rendering the sentence impossible.
*   **Reliance on Smoothing:** Higher-order N-gram models rely heavily on smoothing techniques (like Laplace, Kneser-Ney, etc.) to reallocate probability mass from observed N-grams to unobserved ones. However, even with smoothing, if the sparsity is too extreme, the smoothed probabilities might not be very accurate or discriminative.
*   **Computational Cost:** A larger number of N-grams (even sparse ones) increases memory requirements and computational complexity for training and inference.

In summary, while trigrams capture more linguistic context and can potentially lead to more fluent text generation, the rapidly increasing sparsity problem often makes them impractical without very large corpora and sophisticated smoothing techniques. Bigrams strike a better balance between capturing local context and managing data sparsity for many tasks.

## Summary:

### Data Analysis Key Findings

*   **Sentence Generation by Model Order (N-gram)**:
    *   **Unigram Model**: Sentences generated by the Unigram model (e.g., "<s> deal to he </s>") were largely incoherent and lacked grammatical structure, as words were chosen independently based on individual frequency without considering any sequential context.
    *   **Bigram Model**: Sentences generated by the Bigram model (e.g., "<s> she must pointed animated particulars admire depend father pass belonged able ill. rest invitations serious introducing couple convince catch </s>") showed a significantly higher degree of local fluency and structure due to modeling the probability of a word given the immediately preceding word.

*   **Sentence Generation Across Different Corpora (Bigram)**:
    *   **Gutenberg (Emma) Model**: Sentences generated by the model trained on Jane Austen's "Emma" often contained vocabulary and structures characteristic of a 19th-century novel, exhibiting more natural-sounding English phrases and local grammatical correctness.
    *   **Brown Corpus (News) Model**: Sentences generated by the model trained on the Brown news corpus (e.g., "<s> \<UNK\> of/in this/dt afternoon/nn pay/vb entered/vbd whether/cs hundreds/nns hear/vb charged/vbn kennedy/np fire/nn recovery/nn nine/cd player/np money/nn cut/vb testified/vbd received/vbn </s>") were less coherent and grammatically conventional. This was largely due to the training data containing words explicitly tagged with Part-of-Speech (POS) information, which the model learned to generate.

*   **Language Model Perplexity**:
    *   **Same-Domain Perplexity**: The Gutenberg model achieved a perplexity of 159.5785 when tested on its own domain (Gutenberg corpus).
    *   **Cross-Domain Perplexity**:
        *   The Gutenberg model, when evaluated on the Brown corpus test set, yielded a perplexity of 2098.1836.
        *   The Brown model, when evaluated on the Gutenberg corpus test set, resulted in an even higher perplexity of 2458.2630.
    *   The significantly higher cross-domain perplexity scores demonstrate a strong **domain mismatch**, as models perform poorly when predicting text from a corpus different from their training data due to variations in vocabulary, style, grammar, and N-gram frequencies.

*   **N-gram Sparsity**:
    *   **Trigram (N=3) vs. Bigram (N=2)**: Trigram models face greater sparsity issues compared to Bigram models. This is primarily due to the combinatorial explosion of possible N-grams as `N` increases (e.g., \(V^3\) for trigrams versus \(V^2\) for bigrams, where \(V\) is vocabulary size) and the diminishing likelihood of observing specific longer sequences (longer contexts) in any finite training corpus.
    *   **Consequences**: Sparsity leads to zero probabilities for unobserved N-grams, requiring heavy reliance on smoothing techniques and increasing computational costs.

